In [1]:
%reset -f

In [2]:
import ee
ee.Authenticate()


In [7]:

ee.Initialize(project='ee-andrewfullhart')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

climate_model = 'CCSM4'

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

ndays_months = ee.List([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

def period_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  period_ic = nex_ic.filterDate(start, end)
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  period_ic = period_ic.filter(modelfilter)
  period_ic = period_ic.filter(ee.Filter.eq('model', climate_model))
  period_ic_list = period_ic.toList(period_ic.size())
  def getImageDate(imobj):
    im = ee.Image(imobj)
    return im.date().get('year')
  dates_list = period_ic_list.map(getImageDate)
  dates_list = dates_list.distinct()
  def year_fn(yrobj):
    yr = ee.Number(yrobj)
    start = ee.Date.fromYMD(ee.Number(yr), 1, 1)
    end = ee.Date.fromYMD(ee.Number(yr).add(1), 1, 1)
    yr_ic = period_ic.filterDate(start, end)
    yr_ic_list = yr_ic.toList(yr_ic.size())
    def month_fn(mobj):
      m = ee.Number(mobj)
      pr_im = ee.Image(yr_ic_list.get(m.subtract(1))).select('pr').multiply(86400).multiply(ee.Number(ndays_months.get(m.subtract(1))))
      tx_im = ee.Image(ee.Image(yr_ic_list.get(m.subtract(1))).select('tasmax').subtract(273.15)).multiply(ee.Number(ndays_months.get(m.subtract(1)))).divide(365.25)
      tn_im = ee.Image(ee.Image(yr_ic_list.get(m.subtract(1))).select('tasmin').subtract(273.15)).multiply(ee.Number(ndays_months.get(m.subtract(1)))).divide(365.25)
      tm_im = ee.Image(tx_im.add(tn_im)).divide(2.0).rename('tmean')
      return pr_im.addBands(tm_im)
    converted_ic = ee.ImageCollection(order_months.map(month_fn))
    sum_im = converted_ic.reduce(ee.Reducer.sum()).clip(area_shp)
    return sum_im

  years_ic = ee.ImageCollection(dates_list.map(year_fn))
  mean_im = years_ic.reduce(ee.Reducer.mean())
  return mean_im

mean_ic_list = year_windows.map(period_fn)

yrwin_i = 0

for i in range(year_windows.size().getInfo()):
  start_year = ee.Number(ee.List(year_windows.get(i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i)).select('pr_sum_mean')
  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='accm_{}_{}'.format(str(start_year), str(end_year)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  eimage = ee.Image(mean_ic_list.get(i)).select('tmean_sum_mean')
  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='tmean_{}_{}'.format(str(start_year), str(end_year)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

